In [7]:
import os
import pandas as pd
from polygon import RESTClient
from dotenv import load_dotenv
load_dotenv()

True

### Using python client library

In [8]:
api_key = os.getenv("POLYGON_API_KEY")
client = RESTClient(api_key=api_key)

In [22]:
ticker = "SPY"

# List Aggregates (Bars)
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=60, timespan="minute", from_="2022-01-01", to="2023-06-13", limit=50000):
    aggs.append(a)

print(aggs)

[Agg(open=372.2, high=372.8, low=371.78, close=372.79, volume=35731, vwap=372.3413, timestamp=1665129600000, transactions=294, otc=None), Agg(open=372.73, high=373.65, low=372.73, close=373.41, volume=29814, vwap=373.2497, timestamp=1665133200000, transactions=256, otc=None), Agg(open=373.47, high=373.47, low=372.67, close=373.11, volume=57971, vwap=373.0075, timestamp=1665136800000, transactions=387, otc=None), Agg(open=373.08, high=373.55, low=372.87, close=373.1, volume=142981, vwap=373.2728, timestamp=1665140400000, transactions=1118, otc=None), Agg(open=373.11, high=374.47, low=367.7, close=369.0813, volume=5214026.0, vwap=370.5203, timestamp=1665144000000, transactions=29912, otc=None), Agg(open=369.1, high=369.88, low=365.36, close=365.53, volume=11263726.0, vwap=367.6619, timestamp=1665147600000, transactions=91596, otc=None), Agg(open=365.54, high=367.14, low=364.77, close=366.46, volume=12898209.0, vwap=366.3438, timestamp=1665151200000, transactions=102391, otc=None), Agg(op

In [19]:
def agss_to_df(aggs):
    opens = []
    highs = []
    lows = []
    closes = []
    timestamps = []
    volumes = []

    for agg in aggs:
        opens.append(agg.open)
        highs.append(agg.high)
        lows.append(agg.low)
        closes.append(agg.close)
        timestamps.append(pd.to_datetime(agg.timestamp, unit='ms'))
        volumes.append(agg.volume)

    df_dict = {'timestamp':timestamps, 'open':opens, 'high':highs, 'low':lows, 'close': closes, 'volume':volumes}
    return pd.DataFrame(df_dict).set_index('timestamp')

In [23]:
def get_ohlcv_data(ticker, start_date, end_date, interval):
    """
    Fetch OHLCV data for a given ticker, period, and interval using Polygon API.

    :param ticker: (str) Stock ticker symbol (e.g., 'AAPL', 'TSLA').
    :param period: (str) The duration for which the data is needed (e.g., '1d', '1m', '5y').
    :param interval: (str) The time interval for the data (e.g., 'minute', 'hour', 'day', 'week', 'month').
    :return: (pd.DataFrame) A DataFrame with OHLCV data.
    """
    client = RESTClient(api_key)  # Initialize Polygon API client

    # Period mapping to interval (Polygon uses specific endpoint for each interval type)
    # E.g. for `minute` intervals -> 'minute_aggregates', for 'day' -> 'day_aggregates', etc.
    if interval == 'minute':
        multiplier = 1  # 1-minute interval
        timespan = 'minute'
    elif interval == 'hour':
        multiplier = 60  # 1-hour interval
        timespan = 'minute'
    elif interval == 'day':
        multiplier = 1
        timespan = 'day'
    elif interval == 'week':
        multiplier = 1
        timespan = 'week'
    elif interval == 'month':
        multiplier = 1
        timespan = 'month'
    else:
        raise ValueError("Invalid interval specified. Choose from 'minute', 'hour', 'day', 'week', or 'month'.")

    # Fetch the historical data from Polygon API
    # List Aggregates (Bars)
    aggs = []
    for a in client.list_aggs(ticker=ticker, multiplier=multiplier, timespan=timespan, from_=start_date, to=end_date, limit=50000):
        aggs.append(a)

    # Convert the result to a pandas DataFrame
    df = agss_to_df(aggs)

    return df


In [25]:
get_ohlcv_data(ticker="AAPL", start_date="2000-01-01", end_date="2022-06-30", interval='hour')

BadResponse: {"status":"NOT_AUTHORIZED","request_id":"e7fd57f651adff27d091b5b75bace374","message":"Your plan doesn't include this data timeframe. Please upgrade your plan at https://polygon.io/pricing"}

### Using API url directly

In [16]:
import requests
import pandas as pd
import pandas_ta as ta

# Replace 'YOUR_API_KEY' with your actual API key from Polygon.io
API_KEY = api_key

# Polygon.io API endpoint for historical data
symbol = 'SPY'  # Example stock symbol
timeframe = 'day'  # Daily timeframe
api_url = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/{timeframe}/2020-01-01/2023-01-01?apiKey={API_KEY}'

# Fetch historical data from Polygon.io
response = requests.get(api_url)
data = response.json()